# Importing libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil import parser
from dateutil.relativedelta import relativedelta

# Importing Datasets

In [2]:
demo = pd.read_excel(r"C:\Users\Aradhya\Downloads\data.xlsx", sheet_name='Demo')

In [3]:
demo.head()

,cust_id,fname,lname,city,dob,gender,marital_status
0,89787,vikram,singh,delhi,1980-01-01,m,married
1,139078,ranjit,singh,noida,1982-02-19,m,single
2,67566,anu,sharma,delhi,1984-03-06,f,married
3,67989,amit,shah,noida,1987-06-17,m,divorced
4,56887,vinee,saluja,faridabad,1985-07-06,f,married


In [4]:
north_calls = pd.read_excel(r"C:\Users\Aradhya\Downloads\data.xlsx", sheet_name='Calls_north_centre')
north_calls['Call_from'] = 'North'
north_calls['durationsecs'] = round(north_calls['durationsecs']/60,2)
north_calls.rename(columns={'durationsecs':'durationmins'}, inplace=True)
north_calls.head()

,call_id,account_no,call_type,durationmins,agent_name,called_on,Call_from
0,566777777,11799217050550,in,3.33,amul,2013-04-30,North
1,4555767789,11799217051450,in,0.50,shyam,2013-03-18,North
2,4758799676,12403224437874,out,1.68,lalu,2013-05-01,North
3,6777997,12452031501743,out,0.93,arun,2013-04-17,North
4,123456789,12452031501743,out,1.33,arun,2013-04-21,North


In [5]:
south_east_calls = pd.read_excel(r"C:\Users\Aradhya\Downloads\data.xlsx", sheet_name='Calls_south_east_centre')
south_east_calls['Call_from'] = 'South East'
south_east_calls.head()

,call_id,account_no,call_type,durationmins,agent_name,called_on,Call_from
0,78907986,11756789078985,in,13.3,shamul,2013-04-25,South East
1,68687990,12456789084652,out,1.5,shakhan,2013-04-03,South East
2,57586068,11756789078988,in,14.8,shinee,2013-03-11,South East
3,69799090,11756789078898,in,7.6,sham,2013-03-14,South East
4,86878179,11756789078808,out,8.3,yam,2013-04-29,South East


In [6]:
open_accounts = pd.read_excel(r"C:\Users\Aradhya\Downloads\data.xlsx", sheet_name='Open_accounts')
open_accounts.head()

,account_no,cust_id,account_type,balance,balance_type,open_date
0,11756789078538,56578,fd,65989,+,NaT
1,11756789078628,56887,cc,787,-,2010-05-09
2,11756789078178,56887,cc,4656,-,2010-05-09
3,12456789084653,57579,saving,5657,+,2010-08-09
4,11756789078268,57579,cc,87898,-,2009-12-30


In [7]:
transaction = pd.read_excel(r"C:\Users\Aradhya\Downloads\data.xlsx", sheet_name='Transactions')
transaction.head()

,account_no,tran_id,tran_type,tran_amount,tran_dt
0,12456789084743,10009,debit,40000.0,2013-03-03
1,11756789078448,11119,debit,34000.0,2013-05-05
2,11756789078448,11120,credit,2000.0,2013-05-15
3,11756789078448,11122,debit,2500.0,2013-05-25
4,11756789078448,11139,debit,34000.0,2013-05-30


# Creating New Dataset

In [8]:
final_data = pd.DataFrame()

In [9]:
final_data['cust_id'] = demo['cust_id'].unique()
final_data.sort_values(by=['cust_id'], ignore_index = True)

,cust_id
0,56578
1,56887
2,57579
3,67566
4,67676
5,67687
6,67989
7,89787
8,122233
9,123456


## 1.

### a. No. of open accounts

In [10]:
final_data = pd.merge(final_data, open_accounts.groupby(['cust_id'])['account_no'].count(), on='cust_id', how='left')
final_data.rename(columns={'account_no':'no_of_open_accounts'}, inplace = True)
final_data

,cust_id,no_of_open_accounts
0,89787,4
1,139078,1
2,67566,3
3,67989,4
4,56887,2
5,56578,1
6,57579,3
7,67687,1
8,67676,1
9,123456,1


### b. No. of Inbound Calls

In [11]:
# combining the dataset with information about calls from north and south-east region

all_calls = north_calls.append(south_east_calls)
all_calls = pd.merge(all_calls, open_accounts[['cust_id', 'account_no']], on = 'account_no', how = 'left')
all_calls.head()

C:\Users\Aradhya\AppData\Local\Temp\ipykernel_16568\1509272360.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_calls = north_calls.append(south_east_calls)


,call_id,account_no,call_type,durationmins,agent_name,called_on,Call_from,cust_id
0,566777777,11799217050550,in,3.33,amul,2013-04-30,North,189098.0
1,4555767789,11799217051450,in,0.50,shyam,2013-03-18,North,89787.0
2,4758799676,12403224437874,out,1.68,lalu,2013-05-01,North,67989.0
3,6777997,12452031501743,out,0.93,arun,2013-04-17,North,67566.0
4,123456789,12452031501743,out,1.33,arun,2013-04-21,North,67566.0


In [12]:
final_data = pd.merge(final_data, all_calls[all_calls['call_type']=='in'].groupby(['cust_id']).count()['call_id'], on='cust_id', how='left')
final_data.rename(columns={'call_id':'no_of_inbound_calls'}, inplace = True)
final_data

,cust_id,no_of_open_accounts,no_of_inbound_calls
0,89787,4,5.0
1,139078,1,NaN
2,67566,3,1.0
3,67989,4,2.0
4,56887,2,2.0
5,56578,1,1.0
6,57579,3,5.0
7,67687,1,NaN
8,67676,1,NaN
9,123456,1,NaN


### c. Total Balance

In [13]:
open_accounts['balances'] = (open_accounts['balance_type'] + open_accounts['balance'].astype(str)).astype(int)
final_data = pd.merge(final_data, open_accounts.groupby(['cust_id']).sum()['balances'], on='cust_id', how='left')
final_data.rename(columns={'balances':'total_balance'}, inplace = True)
final_data

C:\Users\Aradhya\AppData\Local\Temp\ipykernel_16568\3048883651.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  final_data = pd.merge(final_data, open_accounts.groupby(['cust_id']).sum()['balances'], on='cust_id', how='left')


,cust_id,no_of_open_accounts,no_of_inbound_calls,total_balance
0,89787,4,5.0,21044
1,139078,1,NaN,-7878
2,67566,3,1.0,795665
3,67989,4,2.0,50090
4,56887,2,2.0,-5443
5,56578,1,1.0,65989
6,57579,3,5.0,-26241
7,67687,1,NaN,-6768
8,67676,1,NaN,455
9,123456,1,NaN,45667


### d & e. Gender and Marital Status

In [14]:
final_data = pd.merge(final_data, demo[['gender','marital_status','cust_id']], on='cust_id', how='left')
final_data

,cust_id,no_of_open_accounts,no_of_inbound_calls,total_balance,gender,marital_status
0,89787,4,5.0,21044,m,married
1,139078,1,NaN,-7878,m,single
2,67566,3,1.0,795665,f,married
3,67989,4,2.0,50090,m,divorced
4,56887,2,2.0,-5443,f,married
5,56578,1,1.0,65989,f,married
6,57579,3,5.0,-26241,m,single
7,67687,1,NaN,-6768,m,single
8,67676,1,NaN,455,f,single
9,123456,1,NaN,45667,f,divorced


### f. Average time spent on calls

In [15]:
final_data = pd.merge(final_data, round(all_calls.groupby(['cust_id']).mean()['durationmins'],2), on='cust_id', how='left')
final_data

C:\Users\Aradhya\AppData\Local\Temp\ipykernel_16568\1466492701.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  final_data = pd.merge(final_data, round(all_calls.groupby(['cust_id']).mean()['durationmins'],2), on='cust_id', how='left')


,cust_id,no_of_open_accounts,no_of_inbound_calls,total_balance,gender,marital_status,durationmins
0,89787,4,5.0,21044,m,married,4.10
1,139078,1,NaN,-7878,m,single,1.49
2,67566,3,1.0,795665,f,married,6.34
3,67989,4,2.0,50090,m,divorced,3.65
4,56887,2,2.0,-5443,f,married,7.80
5,56578,1,1.0,65989,f,married,5.75
6,57579,3,5.0,-26241,m,single,2.94
7,67687,1,NaN,-6768,m,single,1.30
8,67676,1,NaN,455,f,single,1.50
9,123456,1,NaN,45667,f,divorced,4.15


### g. No. of days since last call

In [16]:
all_calls['no_of_days_since_last_call']  = all_calls['called_on'].apply(lambda x: abs(x.date()-date.today()))

In [17]:
final_data = pd.merge(final_data, all_calls.groupby('cust_id').min()['no_of_days_since_last_call'], on='cust_id', how='left')
final_data

,cust_id,no_of_open_accounts,no_of_inbound_calls,total_balance,gender,marital_status,durationmins,no_of_days_since_last_call
0,89787,4,5.0,21044,m,married,4.10,3877 days
1,139078,1,NaN,-7878,m,single,1.49,3895 days
2,67566,3,1.0,795665,f,married,6.34,3875 days
3,67989,4,2.0,50090,m,divorced,3.65,3873 days
4,56887,2,2.0,-5443,f,married,7.80,3883 days
5,56578,1,1.0,65989,f,married,5.75,3878 days
6,57579,3,5.0,-26241,m,single,2.94,3871 days
7,67687,1,NaN,-6768,m,single,1.30,3866 days
8,67676,1,NaN,455,f,single,1.50,3870 days
9,123456,1,NaN,45667,f,divorced,4.15,3872 days


### h. Month on Books

In [18]:
open_accounts['open_date'].fillna('01-01-1900', inplace=True)
open_accounts['months_on_books'] = open_accounts['open_date'].apply(lambda x: relativedelta(date.today(),x.date()).months+(12*relativedelta(date.today(),x.date()).years))
open_accounts

,account_no,cust_id,account_type,balance,balance_type,open_date,balances,months_on_books
0,11756789078538,56578,fd,65989,+,1900-01-01,65989,1487
1,11756789078628,56887,cc,787,-,2010-05-09,-787,162
2,11756789078178,56887,cc,4656,-,2010-05-09,-4656,162
3,12456789084653,57579,saving,5657,+,2010-08-09,5657,159
4,11756789078268,57579,cc,87898,-,2009-12-30,-87898,167
5,12463545858420,57579,fd,56000,+,2010-08-26,56000,159
6,11756789078988,67566,fd,689898,+,2009-05-07,689898,175
7,11756789078808,67566,current,65767,+,2010-01-01,65767,167
8,12452031501743,67566,fd,40000,+,2010-08-02,40000,160
9,11756789078448,67676,saving,455,+,2010-06-06,455,162


In [19]:
final_data = pd.merge(final_data, open_accounts.groupby(['cust_id']).max()['months_on_books'], on='cust_id', how='left')
final_data

,cust_id,no_of_open_accounts,no_of_inbound_calls,total_balance,gender,marital_status,durationmins,no_of_days_since_last_call,months_on_books
0,89787,4,5.0,21044,m,married,4.10,3877 days,177
1,139078,1,NaN,-7878,m,single,1.49,3895 days,164
2,67566,3,1.0,795665,f,married,6.34,3875 days,175
3,67989,4,2.0,50090,m,divorced,3.65,3873 days,177
4,56887,2,2.0,-5443,f,married,7.80,3883 days,162
5,56578,1,1.0,65989,f,married,5.75,3878 days,1487
6,57579,3,5.0,-26241,m,single,2.94,3871 days,167
7,67687,1,NaN,-6768,m,single,1.30,3866 days,160
8,67676,1,NaN,455,f,single,1.50,3870 days,162
9,123456,1,NaN,45667,f,divorced,4.15,3872 days,159


### i. Age of the customer

In [20]:
demo['dob'].fillna('01-01-1900', inplace=True)
demo['age'] = demo['dob'].apply(lambda x: relativedelta(date.today(),x.date()).years)

In [21]:
final_data = pd.merge(final_data, demo.groupby(['cust_id']).sum()['age'], on='cust_id', how='left')
final_data

C:\Users\Aradhya\AppData\Local\Temp\ipykernel_16568\109465850.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  final_data = pd.merge(final_data, demo.groupby(['cust_id']).sum()['age'], on='cust_id', how='left')


,cust_id,no_of_open_accounts,no_of_inbound_calls,total_balance,gender,marital_status,durationmins,no_of_days_since_last_call,months_on_books,age
0,89787,4,5.0,21044,m,married,4.10,3877 days,177,43
1,139078,1,NaN,-7878,m,single,1.49,3895 days,164,41
2,67566,3,1.0,795665,f,married,6.34,3875 days,175,39
3,67989,4,2.0,50090,m,divorced,3.65,3873 days,177,36
4,56887,2,2.0,-5443,f,married,7.80,3883 days,162,38
5,56578,1,1.0,65989,f,married,5.75,3878 days,1487,37
6,57579,3,5.0,-26241,m,single,2.94,3871 days,167,76
7,67687,1,NaN,-6768,m,single,1.30,3866 days,160,53
8,67676,1,NaN,455,f,single,1.50,3870 days,162,123
9,123456,1,NaN,45667,f,divorced,4.15,3872 days,159,40


### j. No. of Debit Transactions

In [22]:
transaction = pd.merge(transaction, open_accounts[['cust_id','account_no']], on='account_no', how='left')

In [23]:
final_data = pd.merge(final_data, transaction[transaction['tran_type']=='debit'].groupby(['cust_id']).count()['tran_id'], on='cust_id', how='left')
final_data.rename(columns={'tran_id':'no_of_debit_trans'}, inplace = True)
final_data

,cust_id,no_of_open_accounts,no_of_inbound_calls,total_balance,gender,marital_status,durationmins,no_of_days_since_last_call,months_on_books,age,no_of_debit_trans
0,89787,4,5.0,21044,m,married,4.10,3877 days,177,43,8.0
1,139078,1,NaN,-7878,m,single,1.49,3895 days,164,41,NaN
2,67566,3,1.0,795665,f,married,6.34,3875 days,175,39,5.0
3,67989,4,2.0,50090,m,divorced,3.65,3873 days,177,36,2.0
4,56887,2,2.0,-5443,f,married,7.80,3883 days,162,38,NaN
5,56578,1,1.0,65989,f,married,5.75,3878 days,1487,37,NaN
6,57579,3,5.0,-26241,m,single,2.94,3871 days,167,76,1.0
7,67687,1,NaN,-6768,m,single,1.30,3866 days,160,53,1.0
8,67676,1,NaN,455,f,single,1.50,3870 days,162,123,3.0
9,123456,1,NaN,45667,f,divorced,4.15,3872 days,159,40,1.0


### k. No. of Credit transactions

In [24]:
final_data = pd.merge(final_data, transaction[transaction['tran_type']=='credit'].groupby(['cust_id']).count()['tran_id'], on='cust_id', how='left')
final_data.rename(columns={'tran_id':'no_of_credit_trans'}, inplace = True)
final_data

,cust_id,no_of_open_accounts,no_of_inbound_calls,total_balance,gender,marital_status,durationmins,no_of_days_since_last_call,months_on_books,age,no_of_debit_trans,no_of_credit_trans
0,89787,4,5.0,21044,m,married,4.10,3877 days,177,43,8.0,6.0
1,139078,1,NaN,-7878,m,single,1.49,3895 days,164,41,NaN,NaN
2,67566,3,1.0,795665,f,married,6.34,3875 days,175,39,5.0,2.0
3,67989,4,2.0,50090,m,divorced,3.65,3873 days,177,36,2.0,2.0
4,56887,2,2.0,-5443,f,married,7.80,3883 days,162,38,NaN,NaN
5,56578,1,1.0,65989,f,married,5.75,3878 days,1487,37,NaN,NaN
6,57579,3,5.0,-26241,m,single,2.94,3871 days,167,76,1.0,2.0
7,67687,1,NaN,-6768,m,single,1.30,3866 days,160,53,1.0,NaN
8,67676,1,NaN,455,f,single,1.50,3870 days,162,123,3.0,1.0
9,123456,1,NaN,45667,f,divorced,4.15,3872 days,159,40,1.0,NaN


### l. Opening Balance for Savings account

In [25]:
final_data = pd.merge(final_data, open_accounts.loc[(open_accounts['account_type']=='saving') | (open_accounts['account_type']=='savings')].groupby(['cust_id']).sum()['balance'], on='cust_id', how='left')
final_data['balance'].fillna(value = -9999999, inplace=True)
final_data.rename(columns={'balance':'opening_balance_for_saving'}, inplace=True)
final_data['opening_balance_for_saving']

C:\Users\Aradhya\AppData\Local\Temp\ipykernel_16568\3829443323.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  final_data = pd.merge(final_data, open_accounts.loc[(open_accounts['account_type']=='saving') | (open_accounts['account_type']=='savings')].groupby(['cust_id']).sum()['balance'], on='cust_id', how='left')


0       21044.0
1    -9999999.0
2    -9999999.0
3       47423.0
4    -9999999.0
5    -9999999.0
6        5657.0
7    -9999999.0
8         455.0
9       45667.0
10      10556.0
11      46567.0
12       7956.0
13       3877.0
14          9.0
Name: opening_balance_for_saving, dtype: float64

In [26]:
open_accounts.loc[(open_accounts['account_type']=='saving') | (open_accounts['account_type']=='savings')]

,account_no,cust_id,account_type,balance,balance_type,open_date,balances,months_on_books
3,12456789084653,57579,saving,5657,+,2010-08-09,5657,159
9,11756789078448,67676,saving,455,+,2010-06-06,455,162
11,11756789078898,67989,savings,656,+,2009-08-09,656,171
12,11756789078358,67989,savings,46767,+,2009-03-03,46767,177
15,11756789078985,89787,saving,8990,+,2010-01-01,8990,167
16,11756789078718,89787,saving,787,+,2009-02-19,787,177
17,17124663638308,89787,saving,4567,+,2010-08-08,4567,160
18,11799217051450,89787,savings,6700,+,2010-09-19,6700,158
20,12456794739408,122233,saving,9,+,2010-07-09,9,160
21,17124663638398,123456,saving,45667,+,2010-08-15,45667,159


In [27]:
open_accounts[open_accounts['account_type']=='saving'].groupby(['cust_id']).sum()['balance']

C:\Users\Aradhya\AppData\Local\Temp\ipykernel_16568\2175096513.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  open_accounts[open_accounts['account_type']=='saving'].groupby(['cust_id']).sum()['balance']


cust_id
57579       5657
67676        455
89787      14344
122233         9
123456     45667
189078     10556
189098       877
199009     46567
1666666     7956
Name: balance, dtype: int64

### m. Opening Balance for Current Account

In [28]:
final_data = pd.merge(final_data, open_accounts[open_accounts['account_type']=='current'].groupby(['cust_id']).sum()['balance'], on='cust_id', how='left')
final_data['balance'].fillna(value = -9999999, inplace=True)
final_data.rename(columns={'balance':'opening_balance_for_current'}, inplace=True)
final_data

C:\Users\Aradhya\AppData\Local\Temp\ipykernel_16568\129330713.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  final_data = pd.merge(final_data, open_accounts[open_accounts['account_type']=='current'].groupby(['cust_id']).sum()['balance'], on='cust_id', how='left')


,cust_id,no_of_open_accounts,no_of_inbound_calls,total_balance,gender,marital_status,durationmins,no_of_days_since_last_call,months_on_books,age,no_of_debit_trans,no_of_credit_trans,opening_balance_for_saving,opening_balance_for_current
0,89787,4,5.0,21044,m,married,4.10,3877 days,177,43,8.0,6.0,21044.0,-9999999.0
1,139078,1,NaN,-7878,m,single,1.49,3895 days,164,41,NaN,NaN,-9999999.0,-9999999.0
2,67566,3,1.0,795665,f,married,6.34,3875 days,175,39,5.0,2.0,-9999999.0,65767.0
3,67989,4,2.0,50090,m,divorced,3.65,3873 days,177,36,2.0,2.0,47423.0,5667.0
4,56887,2,2.0,-5443,f,married,7.80,3883 days,162,38,NaN,NaN,-9999999.0,-9999999.0
5,56578,1,1.0,65989,f,married,5.75,3878 days,1487,37,NaN,NaN,-9999999.0,-9999999.0
6,57579,3,5.0,-26241,m,single,2.94,3871 days,167,76,1.0,2.0,5657.0,-9999999.0
7,67687,1,NaN,-6768,m,single,1.30,3866 days,160,53,1.0,NaN,-9999999.0,6768.0
8,67676,1,NaN,455,f,single,1.50,3870 days,162,123,3.0,1.0,455.0,-9999999.0
9,123456,1,NaN,45667,f,divorced,4.15,3872 days,159,40,1.0,NaN,45667.0,-9999999.0


### n. No. of transactions

In [29]:
final_data = pd.merge(final_data, transaction.groupby(['cust_id']).count()['tran_id'], on='cust_id', how='left')
final_data.rename(columns={'tran_id':'no_of_trans'}, inplace=True)
final_data

,cust_id,no_of_open_accounts,no_of_inbound_calls,total_balance,gender,marital_status,durationmins,no_of_days_since_last_call,months_on_books,age,no_of_debit_trans,no_of_credit_trans,opening_balance_for_saving,opening_balance_for_current,no_of_trans
0,89787,4,5.0,21044,m,married,4.10,3877 days,177,43,8.0,6.0,21044.0,-9999999.0,14.0
1,139078,1,NaN,-7878,m,single,1.49,3895 days,164,41,NaN,NaN,-9999999.0,-9999999.0,NaN
2,67566,3,1.0,795665,f,married,6.34,3875 days,175,39,5.0,2.0,-9999999.0,65767.0,7.0
3,67989,4,2.0,50090,m,divorced,3.65,3873 days,177,36,2.0,2.0,47423.0,5667.0,4.0
4,56887,2,2.0,-5443,f,married,7.80,3883 days,162,38,NaN,NaN,-9999999.0,-9999999.0,NaN
5,56578,1,1.0,65989,f,married,5.75,3878 days,1487,37,NaN,NaN,-9999999.0,-9999999.0,NaN
6,57579,3,5.0,-26241,m,single,2.94,3871 days,167,76,1.0,2.0,5657.0,-9999999.0,3.0
7,67687,1,NaN,-6768,m,single,1.30,3866 days,160,53,1.0,NaN,-9999999.0,6768.0,1.0
8,67676,1,NaN,455,f,single,1.50,3870 days,162,123,3.0,1.0,455.0,-9999999.0,4.0
9,123456,1,NaN,45667,f,divorced,4.15,3872 days,159,40,1.0,NaN,45667.0,-9999999.0,1.0


### o. Amount of Debit Transactions

In [30]:
final_data = pd.merge(final_data, transaction[transaction['tran_type']=='debit'].groupby(['cust_id']).sum()['tran_amount'], on='cust_id', how='left')
final_data.rename(columns={'tran_amount':'amt_of_deb_trans'}, inplace=True)
final_data

C:\Users\Aradhya\AppData\Local\Temp\ipykernel_16568\3603762266.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  final_data = pd.merge(final_data, transaction[transaction['tran_type']=='debit'].groupby(['cust_id']).sum()['tran_amount'], on='cust_id', how='left')


,cust_id,no_of_open_accounts,no_of_inbound_calls,total_balance,gender,marital_status,durationmins,no_of_days_since_last_call,months_on_books,age,no_of_debit_trans,no_of_credit_trans,opening_balance_for_saving,opening_balance_for_current,no_of_trans,amt_of_deb_trans
0,89787,4,5.0,21044,m,married,4.10,3877 days,177,43,8.0,6.0,21044.0,-9999999.0,14.0,66045.0
1,139078,1,NaN,-7878,m,single,1.49,3895 days,164,41,NaN,NaN,-9999999.0,-9999999.0,NaN,NaN
2,67566,3,1.0,795665,f,married,6.34,3875 days,175,39,5.0,2.0,-9999999.0,65767.0,7.0,58000.0
3,67989,4,2.0,50090,m,divorced,3.65,3873 days,177,36,2.0,2.0,47423.0,5667.0,4.0,125000.0
4,56887,2,2.0,-5443,f,married,7.80,3883 days,162,38,NaN,NaN,-9999999.0,-9999999.0,NaN,NaN
5,56578,1,1.0,65989,f,married,5.75,3878 days,1487,37,NaN,NaN,-9999999.0,-9999999.0,NaN,NaN
6,57579,3,5.0,-26241,m,single,2.94,3871 days,167,76,1.0,2.0,5657.0,-9999999.0,3.0,19000.0
7,67687,1,NaN,-6768,m,single,1.30,3866 days,160,53,1.0,NaN,-9999999.0,6768.0,1.0,40000.0
8,67676,1,NaN,455,f,single,1.50,3870 days,162,123,3.0,1.0,455.0,-9999999.0,4.0,70500.0
9,123456,1,NaN,45667,f,divorced,4.15,3872 days,159,40,1.0,NaN,45667.0,-9999999.0,1.0,45667.0


### p. Amount of Credit Transactions

In [31]:
final_data = pd.merge(final_data, transaction[transaction['tran_type']=='credit'].groupby(['cust_id']).sum()['tran_amount'], on='cust_id', how='left')
final_data.rename(columns={'tran_amount':'amt_of_credit_trans'}, inplace=True)
final_data

C:\Users\Aradhya\AppData\Local\Temp\ipykernel_16568\1922616764.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  final_data = pd.merge(final_data, transaction[transaction['tran_type']=='credit'].groupby(['cust_id']).sum()['tran_amount'], on='cust_id', how='left')


,cust_id,no_of_open_accounts,no_of_inbound_calls,total_balance,gender,marital_status,durationmins,no_of_days_since_last_call,months_on_books,age,no_of_debit_trans,no_of_credit_trans,opening_balance_for_saving,opening_balance_for_current,no_of_trans,amt_of_deb_trans,amt_of_credit_trans
0,89787,4,5.0,21044,m,married,4.10,3877 days,177,43,8.0,6.0,21044.0,-9999999.0,14.0,66045.0,27400.0
1,139078,1,NaN,-7878,m,single,1.49,3895 days,164,41,NaN,NaN,-9999999.0,-9999999.0,NaN,NaN,NaN
2,67566,3,1.0,795665,f,married,6.34,3875 days,175,39,5.0,2.0,-9999999.0,65767.0,7.0,58000.0,161556.5
3,67989,4,2.0,50090,m,divorced,3.65,3873 days,177,36,2.0,2.0,47423.0,5667.0,4.0,125000.0,8200.0
4,56887,2,2.0,-5443,f,married,7.80,3883 days,162,38,NaN,NaN,-9999999.0,-9999999.0,NaN,NaN,NaN
5,56578,1,1.0,65989,f,married,5.75,3878 days,1487,37,NaN,NaN,-9999999.0,-9999999.0,NaN,NaN,NaN
6,57579,3,5.0,-26241,m,single,2.94,3871 days,167,76,1.0,2.0,5657.0,-9999999.0,3.0,19000.0,13000.0
7,67687,1,NaN,-6768,m,single,1.30,3866 days,160,53,1.0,NaN,-9999999.0,6768.0,1.0,40000.0,NaN
8,67676,1,NaN,455,f,single,1.50,3870 days,162,123,3.0,1.0,455.0,-9999999.0,4.0,70500.0,2000.0
9,123456,1,NaN,45667,f,divorced,4.15,3872 days,159,40,1.0,NaN,45667.0,-9999999.0,1.0,45667.0,NaN


### q. No of transactions in March 2013

In [32]:
final_data = pd.merge(final_data, transaction[(transaction['tran_dt']>='2013-03-01') & (transaction['tran_dt']<='2013-03-31')].groupby(['cust_id']).count()['tran_id'], on='cust_id', how='left')
final_data.rename(columns={'tran_id':'no_of_trans_in_march_2013'}, inplace=True)
final_data

,cust_id,no_of_open_accounts,no_of_inbound_calls,total_balance,gender,marital_status,durationmins,no_of_days_since_last_call,months_on_books,age,no_of_debit_trans,no_of_credit_trans,opening_balance_for_saving,opening_balance_for_current,no_of_trans,amt_of_deb_trans,amt_of_credit_trans,no_of_trans_in_march_2013
0,89787,4,5.0,21044,m,married,4.10,3877 days,177,43,8.0,6.0,21044.0,-9999999.0,14.0,66045.0,27400.0,3.0
1,139078,1,NaN,-7878,m,single,1.49,3895 days,164,41,NaN,NaN,-9999999.0,-9999999.0,NaN,NaN,NaN,NaN
2,67566,3,1.0,795665,f,married,6.34,3875 days,175,39,5.0,2.0,-9999999.0,65767.0,7.0,58000.0,161556.5,1.0
3,67989,4,2.0,50090,m,divorced,3.65,3873 days,177,36,2.0,2.0,47423.0,5667.0,4.0,125000.0,8200.0,3.0
4,56887,2,2.0,-5443,f,married,7.80,3883 days,162,38,NaN,NaN,-9999999.0,-9999999.0,NaN,NaN,NaN,NaN
5,56578,1,1.0,65989,f,married,5.75,3878 days,1487,37,NaN,NaN,-9999999.0,-9999999.0,NaN,NaN,NaN,NaN
6,57579,3,5.0,-26241,m,single,2.94,3871 days,167,76,1.0,2.0,5657.0,-9999999.0,3.0,19000.0,13000.0,NaN
7,67687,1,NaN,-6768,m,single,1.30,3866 days,160,53,1.0,NaN,-9999999.0,6768.0,1.0,40000.0,NaN,1.0
8,67676,1,NaN,455,f,single,1.50,3870 days,162,123,3.0,1.0,455.0,-9999999.0,4.0,70500.0,2000.0,NaN
9,123456,1,NaN,45667,f,divorced,4.15,3872 days,159,40,1.0,NaN,45667.0,-9999999.0,1.0,45667.0,NaN,NaN


### r. No of transactions in May 2013

In [33]:
final_data = pd.merge(final_data, transaction[(transaction['tran_dt']>='2013-05-01') & (transaction['tran_dt']<='2013-05-31')].groupby(['cust_id']).count()['tran_id'], on='cust_id', how='left')
final_data.rename(columns={'tran_id':'no_of_trans_in_may_2013'}, inplace=True)
final_data

,cust_id,no_of_open_accounts,no_of_inbound_calls,total_balance,gender,marital_status,durationmins,no_of_days_since_last_call,months_on_books,age,no_of_debit_trans,no_of_credit_trans,opening_balance_for_saving,opening_balance_for_current,no_of_trans,amt_of_deb_trans,amt_of_credit_trans,no_of_trans_in_march_2013,no_of_trans_in_may_2013
0,89787,4,5.0,21044,m,married,4.10,3877 days,177,43,8.0,6.0,21044.0,-9999999.0,14.0,66045.0,27400.0,3.0,6.0
1,139078,1,NaN,-7878,m,single,1.49,3895 days,164,41,NaN,NaN,-9999999.0,-9999999.0,NaN,NaN,NaN,NaN,NaN
2,67566,3,1.0,795665,f,married,6.34,3875 days,175,39,5.0,2.0,-9999999.0,65767.0,7.0,58000.0,161556.5,1.0,3.0
3,67989,4,2.0,50090,m,divorced,3.65,3873 days,177,36,2.0,2.0,47423.0,5667.0,4.0,125000.0,8200.0,3.0,NaN
4,56887,2,2.0,-5443,f,married,7.80,3883 days,162,38,NaN,NaN,-9999999.0,-9999999.0,NaN,NaN,NaN,NaN,NaN
5,56578,1,1.0,65989,f,married,5.75,3878 days,1487,37,NaN,NaN,-9999999.0,-9999999.0,NaN,NaN,NaN,NaN,NaN
6,57579,3,5.0,-26241,m,single,2.94,3871 days,167,76,1.0,2.0,5657.0,-9999999.0,3.0,19000.0,13000.0,NaN,NaN
7,67687,1,NaN,-6768,m,single,1.30,3866 days,160,53,1.0,NaN,-9999999.0,6768.0,1.0,40000.0,NaN,1.0,NaN
8,67676,1,NaN,455,f,single,1.50,3870 days,162,123,3.0,1.0,455.0,-9999999.0,4.0,70500.0,2000.0,NaN,4.0
9,123456,1,NaN,45667,f,divorced,4.15,3872 days,159,40,1.0,NaN,45667.0,-9999999.0,1.0,45667.0,NaN,NaN,NaN


### s. Average no of transactions in a month

In [34]:
final_data = pd.merge(final_data, round(transaction.groupby(['cust_id']).count()['tran_id']/3,2), on='cust_id', how='left')
final_data.rename(columns={'tran_id':'avg_no_of_trans'}, inplace=True)
final_data

,cust_id,no_of_open_accounts,no_of_inbound_calls,total_balance,gender,marital_status,durationmins,no_of_days_since_last_call,months_on_books,age,no_of_debit_trans,no_of_credit_trans,opening_balance_for_saving,opening_balance_for_current,no_of_trans,amt_of_deb_trans,amt_of_credit_trans,no_of_trans_in_march_2013,no_of_trans_in_may_2013,avg_no_of_trans
0,89787,4,5.0,21044,m,married,4.10,3877 days,177,43,8.0,6.0,21044.0,-9999999.0,14.0,66045.0,27400.0,3.0,6.0,4.67
1,139078,1,NaN,-7878,m,single,1.49,3895 days,164,41,NaN,NaN,-9999999.0,-9999999.0,NaN,NaN,NaN,NaN,NaN,NaN
2,67566,3,1.0,795665,f,married,6.34,3875 days,175,39,5.0,2.0,-9999999.0,65767.0,7.0,58000.0,161556.5,1.0,3.0,2.33
3,67989,4,2.0,50090,m,divorced,3.65,3873 days,177,36,2.0,2.0,47423.0,5667.0,4.0,125000.0,8200.0,3.0,NaN,1.33
4,56887,2,2.0,-5443,f,married,7.80,3883 days,162,38,NaN,NaN,-9999999.0,-9999999.0,NaN,NaN,NaN,NaN,NaN,NaN
5,56578,1,1.0,65989,f,married,5.75,3878 days,1487,37,NaN,NaN,-9999999.0,-9999999.0,NaN,NaN,NaN,NaN,NaN,NaN
6,57579,3,5.0,-26241,m,single,2.94,3871 days,167,76,1.0,2.0,5657.0,-9999999.0,3.0,19000.0,13000.0,NaN,NaN,1.00
7,67687,1,NaN,-6768,m,single,1.30,3866 days,160,53,1.0,NaN,-9999999.0,6768.0,1.0,40000.0,NaN,1.0,NaN,0.33
8,67676,1,NaN,455,f,single,1.50,3870 days,162,123,3.0,1.0,455.0,-9999999.0,4.0,70500.0,2000.0,NaN,4.0,1.33
9,123456,1,NaN,45667,f,divorced,4.15,3872 days,159,40,1.0,NaN,45667.0,-9999999.0,1.0,45667.0,NaN,NaN,NaN,0.33


### t. No of calls from other region than the branch of the account

In [35]:
df1 = all_calls.loc[(all_calls['account_no'].astype(str).str[:3]=='124') & (all_calls['Call_from']=='South East')].append(
all_calls.loc[((all_calls['account_no'].astype(str).str[:3]=='117') | (all_calls['account_no'].astype(str).str[:3]=='171')) & (all_calls['Call_from']=='North')], ignore_index = True)

C:\Users\Aradhya\AppData\Local\Temp\ipykernel_16568\529103952.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = all_calls.loc[(all_calls['account_no'].astype(str).str[:3]=='124') & (all_calls['Call_from']=='South East')].append(


In [36]:
df1

,call_id,account_no,call_type,durationmins,agent_name,called_on,Call_from,cust_id,no_of_days_since_last_call
0,68687990,12456789084652,out,1.50,shakhan,2013-04-03,South East,139078.0,3901 days
1,575786197,12456789084653,in,3.20,amul,2013-04-22,South East,57579.0,3882 days
2,45456878,12456789084743,out,1.30,vinay,2013-05-08,South East,67687.0,3866 days
3,566777777,11799217050550,in,3.33,amul,2013-04-30,North,189098.0,3874 days
4,4555767789,11799217051450,in,0.50,shyam,2013-03-18,North,89787.0,3917 days
5,9999999999,17124663638308,in,0.57,ram,2013-04-19,North,89787.0,3885 days


In [37]:
final_data = pd.merge(final_data, df1.groupby(['cust_id']).count()['call_id'], on='cust_id', how='left')
final_data.rename(columns={'call_id':'no_of_calls_from_other_region'}, inplace=True)
final_data

,cust_id,no_of_open_accounts,no_of_inbound_calls,total_balance,gender,marital_status,durationmins,no_of_days_since_last_call,months_on_books,age,...,no_of_credit_trans,opening_balance_for_saving,opening_balance_for_current,no_of_trans,amt_of_deb_trans,amt_of_credit_trans,no_of_trans_in_march_2013,no_of_trans_in_may_2013,avg_no_of_trans,no_of_calls_from_other_region
0,89787,4,5.0,21044,m,married,4.10,3877 days,177,43,...,6.0,21044.0,-9999999.0,14.0,66045.0,27400.0,3.0,6.0,4.67,2.0
1,139078,1,NaN,-7878,m,single,1.49,3895 days,164,41,...,NaN,-9999999.0,-9999999.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,67566,3,1.0,795665,f,married,6.34,3875 days,175,39,...,2.0,-9999999.0,65767.0,7.0,58000.0,161556.5,1.0,3.0,2.33,NaN
3,67989,4,2.0,50090,m,divorced,3.65,3873 days,177,36,...,2.0,47423.0,5667.0,4.0,125000.0,8200.0,3.0,NaN,1.33,NaN
4,56887,2,2.0,-5443,f,married,7.80,3883 days,162,38,...,NaN,-9999999.0,-9999999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,56578,1,1.0,65989,f,married,5.75,3878 days,1487,37,...,NaN,-9999999.0,-9999999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,57579,3,5.0,-26241,m,single,2.94,3871 days,167,76,...,2.0,5657.0,-9999999.0,3.0,19000.0,13000.0,NaN,NaN,1.00,1.0
7,67687,1,NaN,-6768,m,single,1.30,3866 days,160,53,...,NaN,-9999999.0,6768.0,1.0,40000.0,NaN,1.0,NaN,0.33,1.0
8,67676,1,NaN,455,f,single,1.50,3870 days,162,123,...,1.0,455.0,-9999999.0,4.0,70500.0,2000.0,NaN,4.0,1.33,NaN
9,123456,1,NaN,45667,f,divorced,4.15,3872 days,159,40,...,NaN,45667.0,-9999999.0,1.0,45667.0,NaN,NaN,NaN,0.33,NaN


### u. No of agents a customer has talked to 

In [38]:
final_data = pd.merge(final_data, all_calls.groupby('cust_id')['agent_name'].nunique(), on='cust_id', how='left')
final_data.rename(columns={'agent_name':'no_of_agents_talked_to'}, inplace=True)
final_data

,cust_id,no_of_open_accounts,no_of_inbound_calls,total_balance,gender,marital_status,durationmins,no_of_days_since_last_call,months_on_books,age,...,opening_balance_for_saving,opening_balance_for_current,no_of_trans,amt_of_deb_trans,amt_of_credit_trans,no_of_trans_in_march_2013,no_of_trans_in_may_2013,avg_no_of_trans,no_of_calls_from_other_region,no_of_agents_talked_to
0,89787,4,5.0,21044,m,married,4.10,3877 days,177,43,...,21044.0,-9999999.0,14.0,66045.0,27400.0,3.0,6.0,4.67,2.0,7
1,139078,1,NaN,-7878,m,single,1.49,3895 days,164,41,...,-9999999.0,-9999999.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2
2,67566,3,1.0,795665,f,married,6.34,3875 days,175,39,...,-9999999.0,65767.0,7.0,58000.0,161556.5,1.0,3.0,2.33,NaN,3
3,67989,4,2.0,50090,m,divorced,3.65,3873 days,177,36,...,47423.0,5667.0,4.0,125000.0,8200.0,3.0,NaN,1.33,NaN,6
4,56887,2,2.0,-5443,f,married,7.80,3883 days,162,38,...,-9999999.0,-9999999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
5,56578,1,1.0,65989,f,married,5.75,3878 days,1487,37,...,-9999999.0,-9999999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
6,57579,3,5.0,-26241,m,single,2.94,3871 days,167,76,...,5657.0,-9999999.0,3.0,19000.0,13000.0,NaN,NaN,1.00,1.0,5
7,67687,1,NaN,-6768,m,single,1.30,3866 days,160,53,...,-9999999.0,6768.0,1.0,40000.0,NaN,1.0,NaN,0.33,1.0,1
8,67676,1,NaN,455,f,single,1.50,3870 days,162,123,...,455.0,-9999999.0,4.0,70500.0,2000.0,NaN,4.0,1.33,NaN,1
9,123456,1,NaN,45667,f,divorced,4.15,3872 days,159,40,...,45667.0,-9999999.0,1.0,45667.0,NaN,NaN,NaN,0.33,NaN,2


#### Cleaning the final Dataset

In [39]:
final_data = final_data.sort_values(by=['cust_id'], ignore_index = True)
final_data['gender'].replace({'f':'female','^m.*':'male'}, regex=True, inplace=True)
final_data['marital_status'].replace('marrried', 'married', inplace=True)
null_value_col = ['no_of_inbound_calls', 'no_of_debit_trans', 'no_of_credit_trans', 'no_of_trans', 'amt_of_deb_trans', 'amt_of_credit_trans', 'no_of_trans_in_march_2013', 'no_of_trans_in_may_2013', 'avg_no_of_trans','no_of_calls_from_other_region']
for i in null_value_col:
    final_data[i].fillna(value=0, inplace=True)
final_data['months_on_books'] = final_data['months_on_books'].replace(1487,np.NaN)
final_data['age'] = final_data['age'].replace(123,np.NaN)
final_data

,cust_id,no_of_open_accounts,no_of_inbound_calls,total_balance,gender,marital_status,durationmins,no_of_days_since_last_call,months_on_books,age,...,opening_balance_for_saving,opening_balance_for_current,no_of_trans,amt_of_deb_trans,amt_of_credit_trans,no_of_trans_in_march_2013,no_of_trans_in_may_2013,avg_no_of_trans,no_of_calls_from_other_region,no_of_agents_talked_to
0,56578,1,1.0,65989,female,married,5.75,3878 days,NaN,37.0,...,-9999999.0,-9999999.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,2
1,56887,2,2.0,-5443,female,married,7.80,3883 days,162.0,38.0,...,-9999999.0,-9999999.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,3
2,57579,3,5.0,-26241,male,single,2.94,3871 days,167.0,76.0,...,5657.0,-9999999.0,3.0,19000.0,13000.0,0.0,0.0,1.00,1.0,5
3,67566,3,1.0,795665,female,married,6.34,3875 days,175.0,39.0,...,-9999999.0,65767.0,7.0,58000.0,161556.5,1.0,3.0,2.33,0.0,3
4,67676,1,0.0,455,female,single,1.50,3870 days,162.0,NaN,...,455.0,-9999999.0,4.0,70500.0,2000.0,0.0,4.0,1.33,0.0,1
5,67687,1,0.0,-6768,male,single,1.30,3866 days,160.0,53.0,...,-9999999.0,6768.0,1.0,40000.0,0.0,1.0,0.0,0.33,1.0,1
6,67989,4,2.0,50090,male,divorced,3.65,3873 days,177.0,36.0,...,47423.0,5667.0,4.0,125000.0,8200.0,3.0,0.0,1.33,0.0,6
7,89787,4,5.0,21044,male,married,4.10,3877 days,177.0,43.0,...,21044.0,-9999999.0,14.0,66045.0,27400.0,3.0,6.0,4.67,2.0,7
8,122233,2,1.0,87887,male,single,2.16,3869 days,170.0,35.0,...,9.0,-9999999.0,2.0,7800.0,9000.0,0.0,0.0,0.67,0.0,3
9,123456,1,0.0,45667,female,divorced,4.15,3872 days,159.0,40.0,...,45667.0,-9999999.0,1.0,45667.0,0.0,0.0,0.0,0.33,0.0,2


## 2.

In [40]:
def age_group(row):
    if (row['age']<=24):
        age_grp = 'very young'
    elif (row['age']>=25 and row['age']<=35):
        age_grp = 'young'
    elif (row['age']>=36 and row['age']<=50):
        age_grp = 'middle age'
    elif (row['age']>=51 and row['age']<100):
        age_grp = 'old'
    else:
        age_grp = 'Unknown'
    return age_grp
final_data['age_group'] = final_data.apply(age_group, axis = 1)
gender_age = pd.crosstab(final_data['age_group'], final_data['gender'])
gender_age

gender,female,male
age_group,,
Unknown,1,0
middle age,5,6
old,0,2
young,0,1


## 3.

In [41]:
def branch_region(col):
    str_col = str(col)
    first3 = str_col[:3]
    region_mapping = {'124':'North','117':'South','171':'East'}
    return region_mapping.get(first3, 'Unknown')
all_calls['branch_region'] = all_calls['account_no'].apply(branch_region)
branch_call = pd.crosstab(all_calls['branch_region'], all_calls['Call_from'])
branch_call

Call_from,North,South East
branch_region,,
East,1,6
North,24,3
South,2,19


### exporting the outputs

In [42]:
with pd.ExcelWriter(r"C:\Users\Aradhya\Downloads\python_case_study.xlsx") as writer:
    final_data.to_excel(writer, sheet_name = "Sheet1", index = False, startrow = 1)
    gender_age.to_excel(writer, sheet_name = "Sheet1", index = True, startrow = 20)
    branch_call.to_excel(writer, sheet_name = "Sheet1", index = True, startrow = 30)